# For historical hurricanes, find closest NEXRAD radar site 

In [180]:
import numpy as np
import pandas as pd
from io import StringIO

## Read all the data
#### hurricane HURDAT2 file trimmed down to 1991-2020 from the 1851-2020 file at https://www.nhc.noaa.gov/data/.
I had to remove N and W from the lat/lon columns, carefully replacing lonE with -lon in a text editor.  

# Ready for the big read-in loop 
#### (debugging steps moved to bottom)

In [181]:
all_hourly_obs = [] 
# a list, not a dataframe: see https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-then-filling-it

In [182]:
datafile = '/Users/bem/Box/MapesGroupUndergradResearch/hurdat2-1991-2020-052921.txt'
fi = open(datafile, 'r') 

while(True): 
    ID = fi.readline()
    if len(ID)<2: 
        break
    parsed = pd.read_csv(StringIO(ID), names=['AL','name','nlines','blank'])
    nlines = parsed.nlines.values[0]
    name = parsed.name.values[0].strip()
    for i in range(nlines): 
        hourlycsv = fi.readline()
        sixhourly = pd.read_csv(StringIO(hourlycsv), usecols=range(8), names=['YMD','HHMM','empty','cat','Nlat','Wlon','ws','slp'])
        sixhourly['name']=name
        all_hourly_obs.append(sixhourly.values)

fi.close()
TChours = pd.DataFrame( np.array(all_hourly_obs).squeeze(), columns=['YMD','HHMM','empty','cat','lat','lon','ws','slp','name'] )
TChours

,YMD,HHMM,empty,cat,lat,lon,ws,slp,name
0,19910629,1200,,LO,25.9,78.0,20,1012,ANA
1,19910629,1800,,LO,25.9,79.0,20,1012,ANA
2,19910630,0,,LO,25.9,80.0,20,1012,ANA
3,19910630,600,,LO,26.0,80.9,20,1012,ANA
4,19910630,1200,,LO,26.2,81.8,20,1012,ANA
...,...,...,...,...,...,...,...,...,...
14608,20201117,1200,,HU,13.7,84.7,75,965,IOTA
14609,20201117,1800,,TS,13.7,85.7,55,988,IOTA
14610,20201118,0,,TS,13.8,86.7,40,1000,IOTA
14611,20201118,600,,TS,13.8,87.8,35,1005,IOTA


In [183]:
# RADAR site locations from https://en.wikipedia.org/wiki/NEXRAD#cite_note-54

sites = pd.read_table('/Users/bem/Box/MapesGroupUndergradResearch/Tools_for_all/WSR88d_locations.txt')

In [184]:
sites[0:5]

,State,Name,Code,Lat,WestLon
0,PR,San Juan,TJUA,18.115600,66.078064
1,ME,Houlton,KCBW,46.039194,67.806603
2,ME,Gray/Portland,KGYX,43.891356,70.256554
3,VT,Burlington,KCXX,44.510994,73.166424
4,MA,Boston\t,KBOX,41.955892,71.136968


In [162]:
# Longitude: how to get the value as a number, from Pandas

TChours.lon.values[0]

78.0

In [163]:
# Site longitude, how to get the value as a number, from Pandas

sites[11:12]['WestLon'].values[0]

74.4108027

# OK, ready for the magic loop
## For each hurricane hour, calculate distance to nearest radar 

Add two columns to hurricanes DataFrame
1. that distance (so we can sort for the 'sweet spot' value)
2. radar ID code

In [173]:
# Loop and collect Code and Distance lists
NearestCode = []
NearestDist = []

for i in range(len(TChours)): 
    # print(TChours[i:i+1]['st'].values[0]) # debugging/testing: print the state
    x = TChours[i:i+1]['lon'].values[0]
    y = TChours[i:i+1]['lat'].values[0]
    min_dist = 1e9 # start with a large value, and find the minimuum
    
    for j in range(len(sites)):
        # print(sites[j:j+1])
        # dx is distance eastward (using cos of latitude correction on sphere)
        dx = (x - sites[j:j+1]['WestLon'].values[0]) * np.cos(y*3.14/180.)
        dy = y - sites[j:j+1]['Lat'].values[0]
        dist2 = dx*dx + dy*dy
        
        if (dist2 < min_dist): 
            # print(sites[j:j+1], '   DISTANCE ', np.sqrt(dist2))
            min_dist = np.sqrt(dist2)
            closest = j 
    #print('') # debugging/testing
    #print('closest station, distance: ')
    #print(sites[closest:closest+1].values[0])
    #print('DISTANCE (km): ', min_dist*111.111)
    #print('')
    
    NearestDist.append(min_dist*111.111) # km 
    NearestCode.append(sites[closest:closest+1]['Code'].values[0])

# Append columns onto the DataFrame

In [174]:
TChours['NearestRadar'] = NearestCode
TChours['NearestDist'] = NearestDist

In [175]:
TChours.sort_values(by='NearestDist',inplace=True)

condition = 'TChours.wid>500'  # a string for filename and also the code to make it happen: safe!

subset = TChours[ eval(cond) ]
subset.reset_index(drop=True, inplace=True) #remove the old index number 


In [176]:
TChours.reset_index(drop=True, inplace=True) #remove the old index number 
TChours

,YMD,HHMM,empty,cat,lat,lon,ws,slp,name,NearestRadar,NearestDist
0,19941115,1300,L,TS,24.6,81.7,45,999,GORDO,KBYX,0.429011
1,20160914,600,,TS,30.5,81.7,40,1011,JULIA,KJAX,1.710969
2,20180914,1800,,HU,34.0,78.4,65,969,FLORENCE,KLTX,2.939565
3,20080726,1800,,LO,31.9,106.7,20,1014,DOLLY,KEPZ,2.993911
4,19970722,600,,TD,33.2,86.8,20,1012,DANNY,KBMX,4.165405
...,...,...,...,...,...,...,...,...,...,...,...
14608,20000825,600,,EX,70.7,4.9,30,994,ALBERTO,TJUA,6261.219431
14609,20040927,1200,,EX,65.0,-10.5,40,986,KARL,TJUA,6332.463654
14610,20050923,0,,EX,68.8,-6.6,30,997,OPHELIA,TJUA,6345.820245
14611,20040927,1800,,EX,65.3,-12.0,35,989,KARL,TJUA,6376.566901


In [177]:
filename = '/Users/bem/Box/MapesGroupUndergradResearch//1991-2020_TChours_byRadarDistances.csv'
TChours.to_csv(filename)

In [178]:
test = pd.read_csv(filename)
test

,Unnamed: 0,YMD,HHMM,empty,cat,lat,lon,ws,slp,name,NearestRadar,NearestDist
0,0,19941115,1300,L,TS,24.6,81.7,45,999,GORDO,KBYX,0.429011
1,1,20160914,600,,TS,30.5,81.7,40,1011,JULIA,KJAX,1.710969
2,2,20180914,1800,,HU,34.0,78.4,65,969,FLORENCE,KLTX,2.939565
3,3,20080726,1800,,LO,31.9,106.7,20,1014,DOLLY,KEPZ,2.993911
4,4,19970722,600,,TD,33.2,86.8,20,1012,DANNY,KBMX,4.165405
...,...,...,...,...,...,...,...,...,...,...,...,...
14608,14608,20000825,600,,EX,70.7,4.9,30,994,ALBERTO,TJUA,6261.219431
14609,14609,20040927,1200,,EX,65.0,-10.5,40,986,KARL,TJUA,6332.463654
14610,14610,20050923,0,,EX,68.8,-6.6,30,997,OPHELIA,TJUA,6345.820245
14611,14611,20040927,1800,,EX,65.3,-12.0,35,989,KARL,TJUA,6376.566901


-------------
# build the URL for Amazon data folder and make it a column 

In [179]:
###### needs to be in this format: 
# https://s3.amazonaws.com/noaa-nexrad-level2/index.html#1997/06/06/KAMA/
# NEEDS ZERO-PADDING # NEEDS TO BE 2 DIGITS


# DEBUGGING LEFTOVERS -- just delete them? 

In [107]:
datafile = '/Users/bem/Box/MapesGroupUndergradResearch/hurdat2-1991-2020-052921.txt'
fi = open(datafile, 'r') 

In [112]:
ID = fi.readline()
ID

'AL021991,            UNNAMED,      6,\n'

In [132]:
len(ID)

38

In [115]:
parsed = pd.read_csv(StringIO(ID), names=['AL','name','nlines','blank'])
parsed

,AL,name,nlines,blank
0,AL021991,UNNAMED,6,NaN


In [116]:
nlines = parsed.nlines.values[0]
nlines

6

In [117]:
name = parsed.name.values[0].strip()
name

'UNNAMED'

In [124]:
for i in range(nlines): 
    hourlycsv = fi.readline()
    sixhourly = pd.read_csv(StringIO(hourlycsv), usecols=range(8), names=['YMD','HHMM','empty','cat','lat','lon','ws','slp'])
    sixhourly['name']=name
    all_hourly_obs.append(sixhourly.values)
    
all_hourly_obs?

Type:        list
String form:
[array([[19910629, 1200, '  ', ' LO', ' 25.9N', '  78.0W', 20, 1012,
           'ANA']], dtype=objec <...> y([[19910817, 600, '  ', ' TS', ' 27.8N', '  76.5W', 45, 998,
           'UNNAMED']], dtype=object)]
Length:      62
Docstring:  
Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.


In [130]:
df = pd.DataFrame( np.array(all_hourly_obs).squeeze(), columns=['YMD','HHMM','empty','cat','lat','lon','ws','slp','name'] )
df

,YMD,HHMM,empty,cat,lat,lon,ws,slp,name
0,19910629,1200,,LO,25.9N,78.0W,20,1012,ANA
1,19910629,1800,,LO,25.9N,79.0W,20,1012,ANA
2,19910630,0,,LO,25.9N,80.0W,20,1012,ANA
3,19910630,600,,LO,26.0N,80.9W,20,1012,ANA
4,19910630,1200,,LO,26.2N,81.8W,20,1012,ANA
...,...,...,...,...,...,...,...,...,...
57,19910816,600,,TD,25.7N,74.9W,25,1012,UNNAMED
58,19910816,1200,,TD,25.9N,75.4W,30,1010,UNNAMED
59,19910816,1800,,TS,26.4N,75.8W,35,1005,UNNAMED
60,19910817,0,,TS,27.1N,76.2W,40,1003,UNNAMED
